In [1]:
import pandas as pd
from meetup import get_meetup_events
from mhub import get_mhub_events
from luma import get_luma_group_events
from common import upload_to_gsheets, get_gsheet_df, create_event_markdown, create_discord_list, scrape_user_events, filter_non_chicago_or_online_events, scrape_events_and_clean_data
from datetime import datetime

#scrape_events_and_clean_data,
#TODO: Add text to each automated pasting to confirm is clean and updates based on the month ect. (1/2 Done just need to be cleaned)
# TODO: Get 1871 Events (1/2 Done just need to be cleaned)
# TODO: Figure out a way to make this format nicely for pasting into slack/discord (1/2 Done just neneds to be cleaned)
# TODO: Filter non-Chicago events from meetup groups (e.g. https://www.meetup.com/pydatachi/events/303000483/) (1/2 Done just need to be cleaned)
# TODO: Split slack messages into multiple markdowns to make life easier (1/2 Just need to be cleaned)
# TODO: Add Organization to Markdown (1/2 Just need to be cleaned)
# TODO: Get lu.ma users Events (1/2)
# TODO: Confirm all gets combined together cleanly (1/2)



#Write a script to update github in format https://github.com/MartinRosenberg/Awesome-Tech-Communities
#(Write a script that will convert the Google Sheet of Communities into a format like https://github.com/MartinRosenberg/Awesome-Tech-Communities)
# TODO: Get Eventbrite Events

# TODO: Make a form to submit events (Google Forms?) 
# TODO: Figure out the list of others that need to be scraped. --> Pull what Driscoll said
# TODO: Make a website/calendar. 
# TODO: Throw this into ChatGPT to simplify the list?
# TODO: Do a Pipeline for this to be updated daily / Weekly. 

In [2]:
organization_df = get_gsheet_df("Organizations")

In [3]:
meetup_groups = organization_df['Meetup'].tolist()
meetup_groups = list(set([x for x in meetup_groups if len(x) > 0]))

luma_groups = organization_df['LuMa'].tolist()
luma_groups = list(set([x for x in luma_groups if len(x) > 0]))

In [4]:
organization_df

,,Organization,Category,Subcategory,Active,Scraper Configured,Website,Event Page,Description,Last In-Person Event,...,LinkedIn,Slack,Discord,Meetup,LuMa,EventBrite,YouTube,GitHub,Facebook,Mastodon
0,1,Chicago Python User Group,Tech Organizations/Groups/Communities,General/Others,TRUE,,https://www.chipy.org/,https://www.meetup.com/_ChiPy_/,"Founded in 2003, Chicago Python User Group is ...",,...,,https://www.chipy.org/pages/slack/,,https://www.meetup.com/_ChiPy_/,,,https://www.youtube.com/channel/UCT372EAC1orBO...,https://github.com/chicagopython/chipy.org,,
1,1,Codecademy + 100Devs Chicago Meetup,Tech Organizations/Groups/Communities,General/Others,FALSE,,https://www.meetup.com/100devs-meetup-group/,https://www.meetup.com/100devs-meetup-group/,"Hello!\nWelcome to 312Coders, a vibrant coding...",,...,,,,https://www.meetup.com/100devs-meetup-group/,,,,,,
2,1,312Coders,Tech Organizations/Groups/Communities,Social,TRUE,,https://www.meetup.com/312-coders/,https://www.meetup.com/312-coders/,"Hello!\nWelcome to 312Coders, a vibrant coding...",,...,,,,https://www.meetup.com/312-coders/,,,,,,
3,1,84.51 Science and Technology Solutions - Chicago,,,TRUE,,https://www.meetup.com/84-51-science-and-techn...,https://www.meetup.com/84-51-science-and-techn...,The purpose of this group is to share exciting...,,...,,,,https://www.meetup.com/84-51-science-and-techn...,,,,,,
4,1,8th Light University,,,TRUE,,https://www.meetup.com/8th-light-university/,https://www.meetup.com/8th-light-university/,8th Light University is a virtual event series...,,...,,,,https://www.meetup.com/8th-light-university/,,,https://www.youtube.com/@8thLightInc,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1,BrowserStack Software Testing meetup group - C...,,,TRUE,,https://www.meetup.com/browserstack-software-t...,https://www.meetup.com/browserstack-software-t...,,,...,,,,https://www.meetup.com/browserstack-software-t...,,,,,,
236,1,ADPList- Chicago,,,TRUE,,https://lu.ma/adplistcommunity?tag=chicago%20%...,https://lu.ma/adplistcommunity?tag=chicago%20%...,,,...,,,,,https://lu.ma/adplistcommunity?tag=chicago%20%...,,,,,
237,1,Startup Valley,,,TRUE,,https://lu.ma/StartupValley?tag=chicago,https://lu.ma/StartupValley?tag=chicago,,,...,,,,,https://lu.ma/StartupValley?tag=chicago,,,,,
238,,Machine Learning at Illinois Tech,,,,,https://lu.ma/user/mliit,https://lu.ma/user/mliit,,,...,,,,,https://lu.ma/user/mliit,,,,,


In [5]:
luma_groups

['https://lu.ma/user/usr-axAqah5w43Nbx7B',
 'https://lu.ma/genai-collective',
 'https://lu.ma/thecomputecollective',
 'https://lu.ma/adplistcommunity?tag=chicago%20%F0%9F%87%BA%F0%9F%87%B8',
 'https://lu.ma/chicago-mlops',
 'https://lu.ma/user/usr-EyRJo0jQJUmLCVw',
 'https://lu.ma/ethchicago',
 'https://lu.ma/user/pmclubiit',
 'https://lu.ma/user/mliit',
 'https://lu.ma/chicagotechcommunity',
 'https://lu.ma/calendar/cal-1qExjhKQejTOW8O',
 'https://lu.ma/itl',
 'https://lu.ma/StartupValley?tag=chicago']

In [6]:
try:
    luma_df = get_luma_group_events(luma_groups, sleep_time=1, verbose=False, debug=True)
except Exception as e:
    print('Failed to get LuMa events')
    print(e)

Skipping user page - https://lu.ma/user/usr-axAqah5w43Nbx7B
'address'
Skipping user page - https://lu.ma/user/usr-EyRJo0jQJUmLCVw
Skipping user page - https://lu.ma/user/pmclubiit
Skipping user page - https://lu.ma/user/mliit


In [7]:
luma_df

,eventName,eventURL,eventStartTime,eventendTime,eventVenueName,eventAddress,eventCity,eventState,groupName,eventGoogleMaps,event_description,datetimeScraped
0,TECHNĒ CHICAGO,https://lu.ma/xs3g4i1h,2025-04-23 08:30:00.000,2025-04-23 13:00:00.000,,Register to See Address,Chicago,IL,ETHChicago Events,,APRIL 23 // 9 - 1 pm : SYMPOSIUM PANELS & GALL...,2025-04-23 14:56:45.788483+00:00
5,☕️ 1 Million Cups Chicago ☕️,https://lu.ma/nca7hs7i,2025-04-23 09:00:00.000,2025-04-23 10:00:00.000,Workbox Chicago - Fulton Market,Workbox Chicago - Fulton Market,Chicago,IL,Chicago Tech Community,https://www.google.com/maps/search/?api=1&quer...,"☕️ 1 Million Cups Chicago☕️\n\n📅 Wednesday, Ap...",2025-04-23 14:56:27.557251+00:00
1,Solana Across America — CHICAGO,https://lu.ma/cj2e58nb,2025-04-23 10:00:00.000,2025-04-23 18:30:00.000,,Register to See Address,Chicago,IL,TECHNĒ CHICAGO,,Hackathon + Happy Hour!\nJoin Helius and Peren...,2025-04-23 14:56:29.132409+00:00
7,FOHE Chicago Spring Happy Hour & Social,https://lu.ma/dl9zhuyt,2025-04-23 17:30:00.000,2025-04-23 20:00:00.000,The Red Lion Pub,The Red Lion Pub,Chicago,IL,Solana Across America — CHICAGO,https://www.google.com/maps/search/?api=1&quer...,"Join us for an evening of networking, collabor...",2025-04-23 14:56:30.442701+00:00
8,🍧Top Shelf Tasting🍧,https://lu.ma/219uhy9s,2025-04-24 18:00:00.000,2025-04-24 22:00:00.000,Guild Row,Guild Row,Chicago,IL,FOHE Chicago Spring Happy Hour & Social,https://www.google.com/maps/search/?api=1&quer...,🍧 Top Shelf Tasting @ Guild Row Social Club 🍧...,2025-04-23 14:56:31.824818+00:00
9,Her Career Story - Speaker Series Event on Bui...,https://lu.ma/hcs-chicago-4-24-25,2025-04-24 18:00:00.000,2025-04-24 20:00:00.000,227 W Monroe St,227 W Monroe St,Chicago,IL,🍧Top Shelf Tasting🍧,https://www.google.com/maps/search/?api=1&quer...,Join us for our Her Career Story Speaker Serie...,2025-04-23 14:56:33.284516+00:00
2,🌳🐶 Web3 Weekends: Park & Pups 🐶🌳,https://lu.ma/ihlb5hom,2025-04-26 10:00:00.000,2025-04-26 12:00:00.000,,Register to See Address,Chicago,IL,Solana Across America — CHICAGO,,🚀Chicago Tech Presents: Web3 Weekends🎉\nChicag...,2025-04-23 14:56:47.469600+00:00
10,Perceptron Social Hour,https://lu.ma/lj2iaykr,2025-04-30 18:00:00.000,2025-04-30 20:00:00.000,Emporium Arcade Bar,Emporium Arcade Bar,Chicago,IL,ML Paper Reading Group,https://www.google.com/maps/search/?api=1&quer...,"Alright, guys, it’s time for all us Turing mac...",2025-04-23 14:57:02.136966+00:00
11,Inside the Loop Biweekly Coffee,https://lu.ma/8t1q90ak,2025-05-01 08:00:00.000,2025-05-01 09:00:00.000,Sawada Coffee,Sawada Coffee,Chicago,IL,Inside the Loop,https://www.google.com/maps/search/?api=1&quer...,"A regular gathering of tech founders, investor...",2025-04-23 14:56:08.056972+00:00
3,🎲 Web3 Weekends: Game Night 🎲,https://lu.ma/j0fckb2i,2025-05-02 18:00:00.000,2025-05-02 20:00:00.000,,Register to See Address,Chicago,IL,🌳🐶 Web3 Weekends: Park & Pups 🐶🌳,,Web3 Weekends: Game Night\nChicago is building...,2025-04-23 14:56:48.758810+00:00


In [8]:
try:
    meetup_df = get_meetup_events(meetup_groups, sleep_time=2, verbose=True, debug=True)
except Exception as e:
    print('Failed to get Meetup events')
    print(e)
try:
    mhub_df = get_mhub_events()
except Exception as e:
    print('Failed to get MHub events')
    print(e)
try:
    luma_df = get_luma_group_events(luma_groups, sleep_time=1, verbose=False, debug=True)
except Exception as e:
    print('Failed to get LuMa events')
    print(e)

0/109 -  https://www.meetup.com/100devs-meetup-group/events/
1/109 -  https://www.meetup.com/chicago-unreal-engine-heroes/events/
0/1 -  https://www.meetup.com/chicago-unreal-engine-heroes/events/307343945/
2/109 -  https://www.meetup.com/chipma/events/
0/1 -  https://www.meetup.com/chipma/events/306946910/
3/109 -  https://www.meetup.com/chitownbio/events/
0/3 -  https://www.meetup.com/chitownbio/events/306903060/
1/3 -  https://www.meetup.com/chitownbio/events/306901870/
2/3 -  https://www.meetup.com/chitownbio/events/306903221/
4/109 -  https://www.meetup.com/code-and-coffee-chicago/events/
0/1 -  https://www.meetup.com/code-and-coffee-chicago/events/306959961/
5/109 -  https://www.meetup.com/chicagogo/events/
6/109 -  https://www.meetup.com/Pumping-Station-One/events/
0/30 -  https://www.meetup.com/pumping-station-one/events/307206331/
1/30 -  https://www.meetup.com/pumping-station-one/events/305320095/
2/30 -  https://www.meetup.com/pumping-station-one/events/307330201/
3/30 -  ht

In [9]:
luma_users = scrape_user_events(luma_groups)

👤 Found 4 user pages

🌐 Opening user page: https://lu.ma/user/usr-axAqah5w43Nbx7B
🔎 Searching for event links...
✅ Found 5 event(s)

🔹 [1/5] Visiting event: https://lu.ma/461o2r8x
📝 Event content snippet:
Presented by
EdTech Institute of Chicago
Subscribe
A professional network of EdTech founders, operators, and investors.
Hosted By
Pete Pellizzari
14 Going
Steve Deutsch, Sasha Pena and 12 others
Contact the Host
Report Event
EdTech Sales Masterclass with Josh Chernikoff
EdTech Institute of Chicago
M ...

🔹 [2/5] Visiting event: https://lu.ma/2kdx0k2i
📝 Event content snippet:
Hosted By
Pete Pellizzari
24 Went
Riddhima Mishra, Tatyana Parks and 22 others
Contact the Host
Report Event
EdTech Institute - February meetup
Hosted by Pete Pellizzari
Feb
20
Thursday, February 20
5:30 PM - 7:30 PM
Moe's Cantina - River North
Chicago, Illinois
Registration
Past Event
This event en ...

🔹 [3/5] Visiting event: https://lu.ma/aovvoiel
📝 Event content snippet:
Hosted By
Pete Pellizzari
63 Went
hamza

In [10]:
luma_users

,user_url,event_url,content
0,https://lu.ma/user/usr-axAqah5w43Nbx7B,https://lu.ma/461o2r8x,Presented by\nEdTech Institute of Chicago\nSub...
1,https://lu.ma/user/usr-axAqah5w43Nbx7B,https://lu.ma/2kdx0k2i,Hosted By\nPete Pellizzari\n24 Went\nRiddhima ...
2,https://lu.ma/user/usr-axAqah5w43Nbx7B,https://lu.ma/aovvoiel,"Hosted By\nPete Pellizzari\n63 Went\nhamza, Ba..."
3,https://lu.ma/user/usr-axAqah5w43Nbx7B,https://lu.ma/v4avgbf2,"Hosted By\nPete Pellizzari\n12 Went\nHuayi He,..."
4,https://lu.ma/user/usr-axAqah5w43Nbx7B,https://lu.ma/clvyc7gk,Hosted By\nPete Pellizzari\nAlex George\nDan F...
5,https://lu.ma/user/usr-EyRJo0jQJUmLCVw,https://lu.ma/w474wi35,Presented by\nChicago City Community\nSubscrib...
6,https://lu.ma/user/usr-EyRJo0jQJUmLCVw,https://lu.ma/mepac4sz,Presented by\nChicago City Community\nSubscrib...
7,https://lu.ma/user/usr-EyRJo0jQJUmLCVw,https://lu.ma/ds7b0qo2,Presented by\nChicago City Community\nSubscrib...
8,https://lu.ma/user/usr-EyRJo0jQJUmLCVw,https://lu.ma/o2ddrv8q,Presented by\nChicago City Community\nSubscrib...
9,https://lu.ma/user/usr-EyRJo0jQJUmLCVw,https://lu.ma/pum1cgpr,Presented by\nChicago City Community\nSubscrib...


In [11]:
luma_event = scrape_user_events(luma_groups)

👤 Found 4 user pages

🌐 Opening user page: https://lu.ma/user/usr-axAqah5w43Nbx7B
🔎 Searching for event links...
✅ Found 5 event(s)

🔹 [1/5] Visiting event: https://lu.ma/461o2r8x
📝 Event content snippet:
Presented by
EdTech Institute of Chicago
Subscribe
A professional network of EdTech founders, operators, and investors.
Hosted By
Pete Pellizzari
14 Going
Steve Deutsch, Sasha Pena and 12 others
Contact the Host
Report Event
EdTech Sales Masterclass with Josh Chernikoff
EdTech Institute of Chicago
M ...

🔹 [2/5] Visiting event: https://lu.ma/2kdx0k2i
📝 Event content snippet:
Hosted By
Pete Pellizzari
24 Went
Riddhima Mishra, Tatyana Parks and 22 others
Contact the Host
Report Event
EdTech Institute - February meetup
Hosted by Pete Pellizzari
Feb
20
Thursday, February 20
5:30 PM - 7:30 PM
Moe's Cantina - River North
Chicago, Illinois
Registration
Past Event
This event en ...

🔹 [3/5] Visiting event: https://lu.ma/aovvoiel
📝 Event content snippet:
Hosted By
Pete Pellizzari
63 Went
hamza

In [27]:
def scrape_events_and_clean_data():
    import os
    import time
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urlparse
    from datetime import datetime
    import pandas as pd
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options

    url = 'https://chiirl.com'
    save_dir = 'event_images'
    os.makedirs(save_dir, exist_ok=True)

    def download_image(url, save_dir):
        if not url:
            return ''
        try:
            parsed_url = urlparse(url)
            filename = os.path.basename(parsed_url.path)
            file_path = os.path.join(save_dir, filename)
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            with open(file_path, 'wb') as f:
                f.write(response.content)
            return file_path
        except Exception as e:
            print(f"Failed to download image {url}: {e}")
            return ''

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    events = soup.select('div.eapp-events-calendar-grid-item')
    print(f"Found {len(events)} events")

    data = []
    for i, event in enumerate(events, 1):
        try:
            print(f"\nExtracting data for event {i}...")

            # Extract Date
            month_elem = event.select_one('.eapp-events-calendar-date-element-month')
            day_elem = event.select_one('.eapp-events-calendar-date-element.day')

            month = month_elem.text.strip() if month_elem else ''
            day = day_elem.text.strip() if day_elem else ''
            date = f"{month} {day}" if month and day else 'No Date'  # Use 'No Date' if date is missing
            print(f"Date: {date}")

            title_elem = event.select_one('.eapp-events-calendar-grid-item-name')
            title = title_elem.text.strip() if title_elem else ''
            print(f"Title: {title}")

            time_elem = event.select_one('.eapp-events-calendar-time-time')
            time_range = time_elem.text.strip() if time_elem else ''
            print(f"Time Range: {time_range}")

            location_elem = event.select_one('.eapp-events-calendar-location-text')
            location = location_elem.text.strip() if location_elem else ''
            print(f"Location: {location}")

            desc_elem = event.select_one('.eapp-events-calendar-grid-item-description')
            description = desc_elem.get_text(separator=' ', strip=True) if desc_elem else ''
            print(f"Description: {description}")

            img_elem = event.select_one('img')
            image_url = img_elem['src'] if img_elem and img_elem.has_attr('src') else ''
            print(f"Image URL: {image_url}")

            image_path = download_image(image_url, save_dir)
            print(f"Image saved at: {image_path}")

            # Append event data
            data.append({
                'Date': date,
                'Title': title,
                'Time': time_range,
                'Location': location,
                'Description': description,
                'Image URL': image_path
            })

        except Exception as e:
            print(f"❌ Error extracting event {i}: {e}")
            continue

    print(f"\n🖼️  Images saved in: {save_dir}")

    # Transform data
    transformed_data = []
    for event in data:
        if not event.get('Date') or not event.get('Title'):
            print(f"⚠️ Event might be missing important fields: {event}")

        transformed_data.append({
            'eventName': event.get('Title', ''),
            'eventURL': url,
            'eventStartTime': event.get('Date', 'No Date'),  # Set to 'No Date' if missing
            'eventendTime': '',
            'eventVenueName': '',
            'eventAddress': event.get('Location', ''),
            'eventCity': 'Chicago',
            'eventState': 'IL',
            'groupName': '',
            'eventGoogleMaps': '',
            'event_description': event.get('Description', ''),
            'datetimeScraped': pd.Timestamp.now().isoformat()
        })

    columns = [
        'eventName', 'eventURL', 'eventStartTime', 'eventendTime', 'eventVenueName',
        'eventAddress', 'eventCity', 'eventState', 'groupName', 'eventGoogleMaps',
        'event_description', 'datetimeScraped'
    ]
    df = pd.DataFrame(transformed_data, columns=columns)

    return df


# Run it
chiirl = scrape_events_and_clean_data()


Found 65 events

Extracting data for event 1...
Date: No Date
Title: TECHNĒ CHICAGO
Time Range: 8:30 AM - 1:00 PM
Location: Chicago, Illinois
Description: RSVP APRIL 23 // 9 - 1 pm : SYMPOSIUM PANELS & GALLERY VIEWING
8:30 am: Doors Open and Registration Begins
9 am : Worlding the Cosmos with Art
Claire Webb, Program Director at the Berggruen Institute, where she leads Future Humans, a research initiative exploring how life, intelligence, and technology might evolve in the long term
9:45 am : ART PRESENTATION - Art on the Moon (and Beyond)
Eduardo Kac, Generating Exploration - Artist
10:15 am : COFFEE & CONVERSATION BREAK
10:45 am : CONVERSATION - How to Fund the Future of Art and Technology
Nancy Kawalek, Founder of STAGE at University of Chicago, exploring quantum futures through performance and scientific collaboration
Aniko Berman, Curator, Advisor, Collector / Director of AOTM Gallery
Moderator: Erika Knierim
11:45 am : FIRESIDE DISCUSSION - Why the Digital Transformation of the A

In [23]:
# Define the target column structure
target_columns = [
    'eventName', 'eventURL', 'eventStartTime', 'eventendTime', 'eventVenueName',
    'eventAddress', 'eventCity', 'eventState', 'groupName', 'eventGoogleMaps',
    'event_description', 'datetimeScraped'
]

# Function to align a DataFrame to the target columns
def align_columns(df, columns, name="DataFrame"):
    try:
        print(f"\n🧩 Aligning columns for {name}")
        if not isinstance(df, pd.DataFrame):
            raise ValueError(f"{name} is not a valid DataFrame")
        
        # Add missing columns with None
        for col in columns:
            if col not in df.columns:
                print(f"⚠️  {name} missing column: {col} — filling with None")
                df[col] = None

        # Return with ordered columns
        return df[columns]
    except Exception as e:
        print(f"❌ Failed to align {name}: {e}")
        return pd.DataFrame(columns=columns)  # Return empty DataFrame with correct structure

# Align all DataFrames with names for traceability
meetup_df_aligned = align_columns(meetup_df, target_columns, "meetup_df")
mhub_df_aligned = align_columns(mhub_df, target_columns, "mhub_df")
luma_df_aligned = align_columns(luma_df, target_columns, "luma_df")
clean_df_aligned = align_columns(luma_users, target_columns, "luma_users")
clean_df_aligned2 = align_columns(luma_event, target_columns, "luma_event")
clean_df_aligned3 = align_columns(chiirl, target_columns, "chiirl")

# Attempt to combine everything with fallback
dfs_to_combine = [
    ("meetup_df", meetup_df_aligned),
    ("mhub_df", mhub_df_aligned),
    ("luma_df", luma_df_aligned),
    ("luma_users", clean_df_aligned),
    ("luma_event", clean_df_aligned2),
    ("chiirl", clean_df_aligned3)
]

valid_dfs = []
for name, df in dfs_to_combine:
    if df is not None and not df.empty:
        valid_dfs.append(df)
    else:
        print(f"⚠️  Skipping {name}: DataFrame is None or empty")

# Try concatenating all valid DataFrames
try:
    combined_df = pd.concat(valid_dfs, ignore_index=True)
    print(f"\n✅ Successfully combined {len(valid_dfs)} DataFrames. Total rows: {len(combined_df)}")
except Exception as e:
    print(f"❌ Error during DataFrame concatenation: {e}")
    combined_df = pd.DataFrame(columns=target_columns)



🧩 Aligning columns for meetup_df

🧩 Aligning columns for mhub_df

🧩 Aligning columns for luma_df

🧩 Aligning columns for luma_users

🧩 Aligning columns for luma_event

🧩 Aligning columns for chiirl
⚠️  Skipping chiirl: DataFrame is None or empty

✅ Successfully combined 5 DataFrames. Total rows: 298


C:\Users\hugos\AppData\Local\Temp\ipykernel_36844\528031085.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(valid_dfs, ignore_index=True)


In [17]:
combined_df

,eventName,eventURL,eventStartTime,eventendTime,eventVenueName,eventAddress,eventCity,eventState,groupName,eventGoogleMaps,event_description,datetimeScraped
0,Chicago Data Night: Adam McElhinney (Uptake),https://www.meetup.com/chicago-data-night/even...,2025-04-23 17:30:00,2025-04-23 19:30:00,"Drive Capital, Fulton East Building, 8th Floor",215 N Peoria Street,Chicago,IL,"Data Science Institute, The University of Chicago",https://www.google.com/maps/search/?api=1&quer...,Please join us for our monthly **Chicago Data ...,2025-04-23 15:07:57.910054+00:00
1,Beyond Earth Day: A Discussion About Protectin...,https://www.meetup.com/chitownbio/events/30690...,2025-04-23 18:00:00,2025-04-23 20:00:00,"Sulzer Regional Library, Chicago Public Library",4455 North Lincoln Avenue,Chicago,IL,ChiTownBio,https://www.google.com/maps/search/?api=1&quer...,"Join AWIS, the Chicago Council on Science and ...",2025-04-23 15:08:48.554077+00:00
2,FI Alumni Panel: Meet Portfolio Companies and ...,https://www.meetup.com/chicago-startup-founder...,2025-04-23 19:00:00,2025-04-23 21:00:00,Online event,5472 N Harlem Ave,Chicago,IL,Chicago Startup Founder 101,https://www.google.com/maps/search/?api=1&quer...,**Alumni of the Founder Institute pre-seed acc...,2025-04-23 15:04:54.908047+00:00
3,Moving to the future with Atlassian – Team25 R...,https://www.meetup.com/chicago-analytics-and-d...,2025-04-24 10:00:00,2025-04-24 11:00:00,Online event,3519 N. Elston Ave,Chicago,IL,Chicago Analytics and Data Science in the Cloud,https://www.google.com/maps/search/?api=1&quer...,It's time to analyze and discuss all the news ...,2025-04-23 15:09:38.683680+00:00
4,The Final Countdown: Syncing Product & Marketing,https://www.meetup.com/chipma/events/306946910/,2025-04-24 17:30:00,2025-04-24 20:30:00,Cognizant Studios,"350 N Orleans St, Chicago, IL 60654",Chicago,IL,Chicago Product Management Association,https://www.google.com/maps/search/?api=1&quer...,**Hosting Company:** Cognizant\n\n**Event Desc...,2025-04-23 15:04:45.416509+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
275,None,None,None,None,None,None,None,None,None,None,None,NaT
276,None,None,None,None,None,None,None,None,None,None,None,NaT
277,None,None,None,None,None,None,None,None,None,None,None,NaT
278,None,None,None,None,None,None,None,None,None,None,None,NaT


In [ ]:
filtered_df = filter_non_chicago_or_online_events(combined_df)

In [ ]:
#not_chicago_or_online_df

In [ ]:
upload_to_gsheets(combined_df, "Events",['eventURL'], verbose=True)

In [ ]:
import pandas as pd
import re
from collections import defaultdict

def is_tech_event(event_description):
    if pd.isna(event_description):
        return False
        
    desc = str(event_description).lower()
    
    # Tech keyword patterns (expanded and optimized)
    tech_patterns = re.compile(r"""
        # Programming/Dev
        \b(tech|developer|coding|programming|software\s*engineering|software\s*dev|engineering|algorithm|data\s*structures?)\b|
        \b(python|ruby|react|kotlin|javascript|java|c\+\+|rust|golang|php|swift|flutter|dart|typescript|rlang)\b|
        
        # AI/ML
        \b(ai\b|artificial\s*intelligence|machine\s*learning|ml\b|deep\s*learning|llm|large\s*language\s*model|generative\s*ai)\b|
        \b(chatgpt|gpt|neural\s*network|computer\s*vision|nlp|natural\s*language\s*processing|tensorflow|pytorch|keras)\b|
        
        # Web/Cloud
        \b(web3|blockchain|crypto\w*|solidity|smart\s*contract|ethereum|cloud|aws|azure|gcp|kubernetes|k8s|docker)\b|
        \b(devops|sre|infrastructure\s*as\s*code|iac|terraform|ansible|jenkins|github\s*actions|ci\/cd)\b|
        
        # Systems
        \b(linux|unix|kernel|embedded|raspberry\s*pi|arduino|iot|cybersecurity|infosec|pentest|red\s*team|blue\s*team)\b|
        \b(networking|sysadmin|distributed\s*systems|scaling|tcp\/ip|http\/?[23]?|websocket|dns|cdn|load\s*balancing)\b|
        
        # Web Dev
        \b(frontend|backend|fullstack|html\w*|css\w*|node\.?js|django|rails|laravel|spring|angular|vue|svelte|graphql)\b|
        
        # Databases
        \b(sql|nosql|postgres|mysql|mariadb|mongodb|redis|cassandra|dynamodb|firestore|neo4j|timescale|influxdb)\b|
        
        # Mobile/Hardware
        \b(mobile|android|ios|react\s*native|3d\s*printing|cnc|arduino|raspberry\s*pi|iot|robotics|vr|ar|metaverse)\b|
        
        # Other Tech
        \b(quantum|open\s*source|foss|git|agile|scrum|api|microservices|hackathon|ctf|meetup|conference|workshop)\b
    """, re.VERBOSE | re.IGNORECASE)

    return bool(tech_patterns.search(desc))

def categorize_tech_event(event_description):
    if pd.isna(event_description):
        return "Other Tech"
        
    desc = str(event_description).lower()
    
    categories = {
        'AI/ML': r'\b(ai\b|artificial\s*intelligence|machine\s*learning|ml\b|deep\s*learning|llm|generative\s*ai|chatgpt|gpt|neural\s*network|computer\s*vision|nlp|tensorflow|pytorch)\b',
        'Web3/Blockchain': r'\b(web3|blockchain|crypto\w*|solidity|smart\s*contract|ethereum|defi|nft|dao|dapp|ipfs|filecoin)\b',
        'Cloud/DevOps': r'\b(cloud|aws|azure|gcp|kubernetes|k8s|docker|devops|sre|infrastructure\s*as\s*code|iac|terraform|ansible|jenkins)\b',
        'Programming': r'\b(python|ruby|javascript|java|c\+\+|rust|golang|php|swift|kotlin|typescript|dart|rlang|programming|coding)\b',
        'Cybersecurity': r'\b(cybersecurity|infosec|pentest|red\s*team|blue\s*team|ctf|ethical\s*hacking|owasp|vulnerability|exploit|malware)\b',
        'Web Dev': r'\b(web\s*dev|frontend|backend|fullstack|html|css|node\.?js|django|rails|laravel|spring|angular|vue|svelte|graphql)\b',
        'Data': r'\b(data\s*science|data\s*engineering|analytics|big\s*data|sql|nosql|postgres|mysql|mongodb|spark|hadoop|databricks)\b',
        'Mobile': r'\b(mobile|android|ios|react\s*native|flutter|xamarin|kotlin\s*multiplatform|swiftui|jetpack\s*compose)\b',
        'Hardware/IoT': r'\b(hardware|electronics|iot|arduino|raspberry\s*pi|3d\s*printing|cnc|robotics|vr|ar|sensor|actuator)\b',
        'Other Tech': r''  # Catch-all
    }
    
    for category, pattern in categories.items():
        if re.search(pattern, desc, re.IGNORECASE):
            return category
    return 'Other Tech'

# Main processing function
def process_tech_events(df):
    # Filter tech events
    tech_mask = df['event_description'].apply(is_tech_event)
    tech_df = df[tech_mask].copy()
    
    # Categorize tech events
    tech_df['tech_category'] = tech_df['event_description'].apply(categorize_tech_event)
    
    return tech_df

# Usage example
tech_events_df = process_tech_events(combined_df)

# Display results
print(f"Found {len(tech_events_df)} tech events out of {len(combined_df)} total events")
print("\nTech Categories Breakdown:")
print(tech_events_df['tech_category'].value_counts())

print(f"\nResults saved to {output_path}")

In [ ]:
tech_events_df

In [ ]:
events_df = get_gsheet_df("Events")
create_event_markdown(events_df, "Chicago Events")

events_df = get_gsheet_df("Events")
create_discord_list(events_df, "Chicago Events")

In [ ]:
import pandas as pd
from datetime import datetime

def format_discord_event_posts(df: pd.DataFrame, max_chars: int = 1900) -> list:
    """
    Format DataFrame events into Discord-ready posts, minimizing character use to reduce total post count.
    """
    discord_posts = []
    current_post = []
    current_length = 0
    
    # Ensure required columns exist
    required_columns = ['eventName', 'eventStartTime', 'eventVenueName', 'groupName']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")
    
    # Sort by event time
    df = df.sort_values('eventStartTime')
    
    for _, row in df.iterrows():
        # Gather event details compactly
        event_name = row['eventName'] if pd.notna(row['eventName']) else "Event"
        org = row['groupName'] if pd.notna(row['groupName']) else "N/A"
        start_time = format_datetime(row['eventStartTime']) if pd.notna(row['eventStartTime']) else "N/A"
        venue = row['eventVenueName'] if pd.notna(row['eventVenueName']) else "N/A"
        event_url = row['eventURL'] if 'eventURL' in df.columns and pd.notna(row['eventURL']) else ""
        
        # One-liner event summary (condensed)
        event_line = f"- {event_name} | {start_time} | {org} | {venue}"
        if event_url:
            event_line += f" | {event_url}"

        # Add newline character
        event_line += "\n"
        
        # Check if adding this line exceeds the limit
        if current_length + len(event_line) > max_chars:
            discord_posts.append("".join(current_post))
            current_post = [event_line]
            current_length = len(event_line)
        else:
            current_post.append(event_line)
            current_length += len(event_line)
    
    if current_post:
        discord_posts.append("".join(current_post))
    
    return discord_posts

def format_datetime(dt) -> str:
    """Compact datetime format for brevity"""
    if isinstance(dt, str):
        try:
            dt = pd.to_datetime(dt)
        except:
            return dt
    if isinstance(dt, pd.Timestamp):
        return dt.strftime('%b %d %I:%M%p')  # e.g., "Apr 12 05:30PM"
    return str(dt)

def save_posts_to_file(posts: list, filename: str = "discord_posts.txt"):
    """Save formatted posts to a text file"""
    with open(filename, 'w', encoding='utf-8') as f:
        for i, post in enumerate(posts, 1):
            f.write(f"=== POST {i} ===\n")
            f.write(post)
            f.write("\n")
    print(f"Saved {len(posts)} posts to {filename}")

# Example usage
if __name__ == "__main__":
    # Example: combined_df = pd.read_csv("events.csv")
    posts = format_discord_event_posts(combined_df)
    save_posts_to_file(posts)


In [ ]:
import re
import json
from datetime import datetime

def md_to_slack_blocks(md_content):
    MAX_BLOCKS_PER_MESSAGE = 50
    all_chunks = []
    current_blocks = []

    # Helper to flush current_blocks into all_chunks
    def flush_blocks():
        if current_blocks:
            all_chunks.append({"blocks": list(current_blocks)})
            current_blocks.clear()

    # Split events by the ### marker
    events = re.split(r'### \S+', md_content)[1:]

    # Start with header
    header = {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": "Upcoming Tech Events in Chicago",
            "emoji": True
        }
    }
    divider = {"type": "divider"}
    current_blocks.extend([header, divider])

    for event in events:
        try:
            # Extract details
            title_match = re.search(r'\[([^\]]+)\]\(([^)]+)\)', event)
            org_match = re.search(r'\*Organization:\*\* ([^\n]+)', event)
            when_match = re.search(r'\*When:\*\* ([^\n]+)', event)
            where_match = re.search(r'\*Where:\*\* ([^\n]+)', event)
            details_match = re.search(r'\*Details:\*\s*> ([^\n]+)', event, re.DOTALL)

            if not title_match:
                continue

            title = title_match.group(1)
            url = title_match.group(2)
            org = org_match.group(1).strip() if org_match else "Not specified"
            when = when_match.group(1).strip() if when_match else "Date not specified"
            where = where_match.group(1).strip() if where_match else "Location not specified"
            details = details_match.group(1).strip().replace('\n', ' ') if details_match else "No details provided"

            # Try date formatting
            try:
                dt = datetime.strptime(when, "%A, %B %d @ %I:%M%p")
                when = dt.strftime("%a %b %d at %I:%M %p")
            except ValueError:
                pass

            event_block = {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": f"*<{url}|{title}>*\n:office: *Org:* {org}\n:calendar: *When:* {when}\n:round_pushpin: *Where:* {where}\n\n> {details[:200]}{'...' if len(details) > 200 else ''}"
                },
                "accessory": {
                    "type": "button",
                    "text": {
                        "type": "plain_text",
                        "text": "RSVP",
                        "emoji": True
                    },
                    "value": f"event_{title.lower().replace(' ', '_')}",
                    "url": url,
                    "action_id": "rsvp_button"
                }
            }

            # Check space before adding 2 blocks (section + divider)
            if len(current_blocks) + 2 > MAX_BLOCKS_PER_MESSAGE:
                flush_blocks()
                current_blocks.extend([header, divider])  # New chunk starts with header again

            current_blocks.append(event_block)
            current_blocks.append(divider)

        except Exception as e:
            print(f"Error processing event: {e}")
            continue

    # Final "View All Events" button
    final_button = {
        "type": "actions",
        "elements": [
            {
                "type": "button",
                "text": {
                    "type": "plain_text",
                    "text": "View All Events",
                    "emoji": True
                },
                "value": "view_all",
                "url": "https://example.com/events",
                "action_id": "view_all_button"
            }
        ]
    }

    # Make sure we have space for the button
    if len(current_blocks) + 1 > MAX_BLOCKS_PER_MESSAGE:
        flush_blocks()
        current_blocks.extend([header, divider])

    current_blocks.append(final_button)
    flush_blocks()

    return all_chunks

# Example usage
if __name__ == "__main__":
    try:
        with open('Chicago Events.md', 'r', encoding='utf-8') as f:
            md_content = f.read()

        slack_blocks_chunks = md_to_slack_blocks(md_content)

        # Print each chunk as a separate Slack message
        for i, chunk in enumerate(slack_blocks_chunks):
            print(f"\n--- Message {i+1} ---\n")
            print(json.dumps(chunk, indent=2))

    except FileNotFoundError:
        print("Error: 'Chicago Events.md' file not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
print(meetup_df.columns)
print(mhub_df.columns)
print(luma_df.columns)
